In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os.path as osp
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle
IMG_DIR = '/home/chec/data/open-images/train/imgs'
MASK_DIR = '/home/chec/data/open-images/masks/train'
DATA_DIR = '/home/chec/data/open-images/segmentation'

In [3]:
df_des = pd.read_csv(osp.join(DATA_DIR, 'challenge-2019-classes-description-segmentable.csv'), names=['class', 'name'])
df_des.head()

,class,name
0,/m/01bms0,Screwdriver
1,/m/03jbxj,Light switch
2,/m/0jy4k,Doughnut
3,/m/09gtd,Toilet paper
4,/m/01j5ks,Wrench


In [4]:
class_dict = df_des.set_index('class').to_dict()['name']

In [44]:
def get_class_info():
    with open(osp.join(DATA_DIR, 'challenge-2019-label300-segmentable-hierarchy.json'), 'r') as f:
        h = json.load(f)
    leaf_classes = []
    parent_classes = []
    parent_dict = {}

    def travel(root, parent=[]):
        if 'Subcategory' in root:
            parent_classes.append(root['LabelName'])
            for c in root['Subcategory']:
                travel(c, parent+[root['LabelName']])
        else:
            leaf_classes.append(root['LabelName'])
            parent_dict[root['LabelName']] = parent

    travel(h)

    parent_classes.remove('/m/0bl9f')
    for k in parent_dict:
        parent_dict[k].remove('/m/0bl9f')
        #if parent_dict[k] and k in leaf_classes:
        #    print(k, parent_dict[k])
        #    print(class_dict[k], [class_dict[x] for x in parent_dict[k]])
    return set(leaf_classes), set(parent_classes), parent_dict

In [45]:
leaf_classes, parent_classes, parent_dict = get_class_info()

In [46]:
len(set(parent_classes))

25

In [39]:
df_masks = pd.read_csv(osp.join(DATA_DIR, 'challenge-2019-train-segmentation-masks.csv'))
df_masks.head()

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
0,ccead8c5906e6e11_m09ddx_a881dc5b.png,ccead8c5906e6e11,/m/09ddx,a881dc5b,0.319531,0.657812,0.328271,0.738318,0.88407,0.62748 0.63862 0;0.44461 0.72795 1;0.44990 0....
1,88e582a7b14e34a8_m039xj__6133896f.png,88e582a7b14e34a8,/m/039xj_,6133896f,0.756250,0.810625,0.326797,0.442577,0.76044,0.76475 0.42231 0;0.79720 0.32467 1;0.76502 0....
2,280849e5239012cb_m0342h_c420399e.png,280849e5239012cb,/m/0342h,c420399e,0.000000,0.513950,0.600586,0.739258,0.88397,0.00690 0.67839 1;0.11382 0.62844 0;0.08955 0....
3,6533d5bbc139857d_m0342h_9466984b.png,6533d5bbc139857d,/m/0342h,9466984b,0.115625,0.312500,0.414634,0.605066,0.57796,0.17948 0.51202 1;0.22844 0.50576 0;0.17219 0....
4,74362934ae874b7d_m083wq_0e0c104e.png,74362934ae874b7d,/m/083wq,0e0c104e,0.006250,0.081875,0.470833,0.635833,0.83177,0.00726 0.56089 1;0.06891 0.61937 1;0.07646 0....


In [47]:
df_masks.loc[df_masks.LabelName.isin(parent_classes)].shape

(1296517, 10)

In [35]:
import random
def get_parent_label(label_name):
    if label_name in parent_classes:
        return label_name
    elif label_name in leaf_classes:
        if parent_dict[label_name]:
            return random.choice(parent_dict[label_name])
        else:
            return 'none'

In [41]:
df_masks.LabelName = df_masks.LabelName.map(lambda x: get_parent_label(x))
df_masks.head()

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
0,ccead8c5906e6e11_m09ddx_a881dc5b.png,ccead8c5906e6e11,/m/015p6,a881dc5b,0.319531,0.657812,0.328271,0.738318,0.88407,0.62748 0.63862 0;0.44461 0.72795 1;0.44990 0....
1,88e582a7b14e34a8_m039xj__6133896f.png,88e582a7b14e34a8,none,6133896f,0.756250,0.810625,0.326797,0.442577,0.76044,0.76475 0.42231 0;0.79720 0.32467 1;0.76502 0....
2,280849e5239012cb_m0342h_c420399e.png,280849e5239012cb,none,c420399e,0.000000,0.513950,0.600586,0.739258,0.88397,0.00690 0.67839 1;0.11382 0.62844 0;0.08955 0....
3,6533d5bbc139857d_m0342h_9466984b.png,6533d5bbc139857d,none,9466984b,0.115625,0.312500,0.414634,0.605066,0.57796,0.17948 0.51202 1;0.22844 0.50576 0;0.17219 0....
4,74362934ae874b7d_m083wq_0e0c104e.png,74362934ae874b7d,none,0e0c104e,0.006250,0.081875,0.470833,0.635833,0.83177,0.00726 0.56089 1;0.06891 0.61937 1;0.07646 0....


In [37]:
df_masks.loc[df_masks.LabelName!='none'].shape

(1296517, 10)

In [42]:
df_train = df_masks.loc[df_masks.LabelName!='none']

In [43]:
df_train.LabelName.value_counts()

/m/01g317     634241
/m/0k4j       166635
/m/0c9ph5     100067
/m/07mhn       68365
/m/015p6       57049
/m/0138tl      52553
/m/01lrl       42508
/m/0271t       42213
/m/06msq       30185
/m/02dl1y      22791
/m/0k5j        18433
/m/0ch_cf      13854
/m/018xm        6797
/m/06bt6        6750
/m/02crq1       6127
/m/0hf58v5      6019
/m/0dv77        4718
/m/01mqdt       4520
/m/0l515        2540
/m/0174n1       2483
/m/02wv6h6      1912
/m/0dv9c        1655
/m/01dws        1428
/m/09dzg        1417
/m/01x3z        1257
Name: LabelName, dtype: int64